<a href="https://colab.research.google.com/github/BohdanPetryshyn/code-llama-fim-fine-tuning/blob/main/code_llama_fim_fine_tuning_inference_and_merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Llama FIM fine-tuning inference and merging

If you found a problem with this notebook, please report it in the original GitHub [repo](https://github.com/BohdanPetryshyn/code-llama-fim-fine-tuning?tab=readme-ov-file) as an issue.

## Install dependenices


In [1]:
!git clone https://github.com/BohdanPetryshyn/code-llama-fim-fine-tuning.git repo

Cloning into 'repo'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 37 (delta 16), reused 27 (delta 10), pack-reused 0
Receiving objects: 100% (37/37), 60.53 KiB | 5.04 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [2]:
!pip install ninja
!ninja --version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 5.8 MB/s eta 0:00:00
1.11.1.git.kitware.jobserver-1


In [3]:
%cd repo
!pip install -r requirements.txt

/content/repo
  Cloning https://github.com/unslothai/unsloth.git (to revision 27fa021a7bb959a53667dd4e7cdb9598c207aa0d) to /tmp/pip-install-omnjgycd/unsloth_0b19d6cab6e54f449cf599c4b4ea808e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-omnjgycd/unsloth_0b19d6cab6e54f449cf599c4b4ea808e
  Running command git rev-parse -q --verify 'sha^27fa021a7bb959a53667dd4e7cdb9598c207aa0d'
  Running command git fetch -q https://github.com/unslothai/unsloth.git 27fa021a7bb959a53667dd4e7cdb9598c207aa0d
  Running command git checkout -q 27fa021a7bb959a53667dd4e7cdb9598c207aa0d
  Resolved https://github.com/unslothai/unsloth.git to commit 27fa021a7bb959a53667dd4e7cdb9598c207aa0d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 6.3 MB/s eta 0:00:00
     

## Load base Code Llama model

In [4]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)

base_model_id = "codellama/CodeLlama-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [5]:
import torch

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=None,
    device_map=None,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)

model.cuda()

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

## Load fine-tuned adapter

In [6]:
from peft import PeftModel

# Replace with your adapter ID
adapter_id = "BohdanPetryshyn/codellama-7b-openapi-completion-ctx-lvl-fim-05-spm"
# Latest revision
revision = None
model = PeftModel.from_pretrained(model, adapter_id, revision=revision, adapter_name="my-adapter")
model.set_adapter("my-adapter")

adapter_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/320M [00:00<?, ?B/s]

## Inference test

In [7]:
def get_completion(prefix, suffix, prompt = None):
    if prompt == None:
      prompt = f"""<PRE> {prefix} <SUF>{suffix} <MID>"""
    if not isinstance(prompt, list):
      prompt = tokenizer(prompt).input_ids

    model.eval()
    outputs = model.generate(
        input_ids=torch.tensor([prompt]).cuda(),
        max_new_tokens=256,
    )
    return (outputs, tokenizer.batch_decode(outputs, skip_special_tokens=False)[0])

In [8]:
prefix = """
  /special-events:
    post:
      summary: Create special events
      description: Creates a new special event for the museum.
      operationId: createSpecialEvent
      tags:
        - Events
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/CreateSpecialEventRequest'
            examples:
              default_example:
                $ref: '#/components/examples/CreateSpecialEventRequestExample'
      responses:
        '200':
          description: Success
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/SpecialEventResponse'
              examples:
                default_example:
                  $ref: '#/components/examples/CreateSpecialEventResponseExample'
        '400':
          description: Bad request
        '404':
          description: Not found
    get:
      summary: List special events
      description: Return a list of upcoming special events at the museum.
      operationId: listSpecialEvents
      tags:
        - Events
      parameters:"""

suffix = """  /special-events/{eventId}:
    get:
      summary: Get special event
      description: Get details about a special event.
      operationId: getSpecialEvent
      tags:
        - Events
      parameters:
        - $ref: '#/components/parameters/EventId'
      responses:
        '200':
          description: Success
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/SpecialEventResponse'
              examples:
                default_example:
                  $ref: '#/components/examples/GetSpecialEventResponseExample'
        '400':
          description: Bad request
        '404':
          description: Not found"""

tokens, result = get_completion(prefix, suffix)

print(result)
print([tokenizer.decode(token) for token in tokens[0]])
print(tokens[0].tolist())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> <PRE> 
  /special-events:
    post:
      summary: Create special events
      description: Creates a new special event for the museum.
      operationId: createSpecialEvent
      tags:
        - Events
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/CreateSpecialEventRequest'
            examples:
              default_example:
                $ref: '#/components/examples/CreateSpecialEventRequestExample'
      responses:
        '200':
          description: Success
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/SpecialEventResponse'
              examples:
                default_example:
                  $ref: '#/components/examples/CreateSpecialEventResponseExample'
        '400':
          description: Bad request
        '404':
          description: Not found
    get:
      summary: List special events
    

## Merge and upload the model

In [9]:
from huggingface_hub import notebook_login

notebook_login()

In [10]:
# Replace with your desired merged model ID
merged_model_id = "code-llama-openapi-completion"

model = model.merge_and_unload()
model.save_pretrained(merged_model_id)
model.push_to_hub(merged_model_id)

[2024-06-15 09:57:04,444] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BohdanPetryshyn/code-llama-openapi-completion/commit/066b109f08930ee89b44f097e6a0a98201f2e9d8', commit_message='Upload LlamaForCausalLM', commit_description='', oid='066b109f08930ee89b44f097e6a0a98201f2e9d8', pr_url=None, pr_revision=None, pr_num=None)

### Upload Code Llama tokenizer to the model repository

To make the merged model fully functional withing the HF Inference Endponits platform, we have to add a tokenizer to the repository. For some reason, the following approach results in the model not generating `<EOT>` token during inference:

```
tokenizer.push_to_hub(merged_model_id)
```

So we are copying the original Code Llama tokenizer files manually using git CLI.

In [11]:
# Replace with your Hugging Face account name
full_merged_model_id = f"BohdanPetryshyn/{merged_model_id}"

%cd /content

# Clone the merged repo. Skipping large files
!export GIT_LFS_SKIP_SMUDGE=1 && git clone https://huggingface.co/{full_merged_model_id} merged_model

# Clone the original Code Llama repo to copy the tokenizer files
!export GIT_LFS_SKIP_SMUDGE=1 && git clone https://huggingface.co/codellama/CodeLlama-7b-hf original_code_llama
!cd original_code_llama && git lfs pull --include "tokenizer*"

%cd merged_model

!cp ../original_code_llama/special_tokens_map.json .
!cp ../original_code_llama/tokenizer* .

!git config --global user.email "code-llama-fim-fine-tuning@colab.research.google.com"
!git config --global user.name "Code Llama FIM Fine Tuning by Bohdan Petryshyn"
!git add .
!git commit -m "Add tokenizer from the original Code Llama model"
!git push origin main

/content
Cloning into 'merged_model'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 3), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (23/23), 485.16 KiB | 3.76 MiB/s, done.
Cloning into 'original_code_llama'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 100 (delta 35), reused 20 (delta 20), pack-reused 55 (from 1)
Receiving objects: 100% (100/100), 505.60 KiB | 2.44 MiB/s, done.
Resolving deltas: 100% (35/35), done.
/content/merged_model
[main ed459f2] Add tokenizer from the original Code Llama model
 4 files changed, 93478 insertions(+)
 create mode 100644 special_tokens_map.json
 create mode 100644 tokenizer.json
 create mode 100644 tokenizer.model
 create mode 100644 tokenizer_config.json
Uploading LFS objects: 100% (1/1), 500 KB | 0 B/s, done.
